In [1]:
# Imports
import numpy as np
import pandas as pd

import langchain
import torch

import huggingface_hub

# for env vars
import settings

# check langchain version
langchain.__version__

'0.0.298'

In [2]:
# Read the example dataset
df = pd.read_csv("example_data/example.csv")

# See if it loaded correctly
df.head(5)

,FixTypos,Chunk
0,catt,jeep
1,hunter-,jeeping
2,huner,"jeep, vehicle: 4wd"
3,-alzheimer's patient,horseback
4,snowborder,horseback riding


In [3]:
# Ok, it loaded. Now let's check if the columns were read correctly
df.columns

Index(['FixTypos', 'Chunk'], dtype='object')

In [4]:
df["FixTypos"].unique()

array(['catt', 'hunter-', 'huner', "-alzheimer's patient", 'snowborder',
       'snowboarder', 'missing person', 'other-camper', 'bicylist',
       'dog_', 'aircraft   e-', 'flood***victimss'], dtype=object)

In [5]:
# these instruct-likes are more about text completion rather than responding to inputs
# they only produce desirable behavior when inference is NOT done in batches but instead is done throughout the entire list 
instruct_like = [  
    "HuggingFaceH4/zephyr-7b-beta",
    "tiiuae/falcon-7b-instruct",
    "mistralai/Mistral-7B-Instruct-v0.1",
    "mistralai/Mistral-7B-v0.1",
    "declare-lab/flan-alpaca-large",
    "bigcode/starcoder",
    "bigscience/bloom"
]

timeouts = [
    "01-ai/Yi-34B",
    "01-ai/Yi-6B",
    "openchat/openchat_3.5"
]

exihibits_somewhat_desired_behavior = [
    "google/flan-t5-xxl"
]

In [9]:
# Nodes
from chains import FewShotLLM
from prompts import typofix, chunker

# Models
from langchain.llms import HuggingFaceHub
#from ml_models.apis import HUGGINGFACE_APIs as HF
#from ml_models.models import HuggingFaceLLM # somewhat deprecated


#typofix_ptuner = FewShotLLM(HuggingFaceLLM(api=HF.Falcon_7b_Instruct), typofix.FEW_SHOT_PROMPT)
#chunker_ptuner = FewShotLLM(HuggingFaceLLM(api=HF.Falcon_7b_Instruct), chunker.FEW_SHOT_PROMPT)
default_kwargs = {"temperature": 0.5, "max_length": 500}  # 0.0 = most determinstic, 1.0 = most stochastic 


#typofix_ptuner = FewShotLLM(HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs=default_kwargs), typofix.FEW_SHOT_PROMPT)
typofix_ptuner = FewShotLLM(HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs=default_kwargs), 
                            typofix.FEW_SHOT_PROMPT, wrap_input=typofix.WRAP_INPUT)
chunker_ptuner = FewShotLLM(HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.1", model_kwargs=default_kwargs),
                            chunker.FEW_SHOT_PROMPT, wrap_input=chunker.WRAP_INPUT)

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/tiiuae/falcon-7b-instruct (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f4e77f86cd0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))"), '(Request ID: 09c504f2-6f5c-4abb-bf21-2efdff446d9e)')

In [ ]:
from chains import ChatChain

# Putting it all together
chain = typofix_ptuner.chain | chunker_ptuner.chain
final_chain = ChatChain(chain)  # for production

In [8]:
import data_cleaning as dcl

base_tests = ['earth%^&quakeee', 'aaalligatoorr', 'camp-']
propagated_tests = ['earthquake', 'aaalligatoorr', 'camp']

# Time to try!
#df_cleaned = dcl.clean_column(typofix_ptuner, df, "FixTypos")
#dcl.inference_clean(typofix_ptuner, df["FixTypos"].unique(), batch_size=3)
#print(dcl.safe_chain_inference(typofix_ptuner, df["FixTypos"].unique()))
print(dcl.safe_chain_inference(typofix_ptuner, propagated_tests, batch_size=2))
#print(dcl.raw_chain_inference(typofix_ptuner, propagated_tests))

# Next test: Test on a validation set
# Next step: validator for spellchecking
# Maybe have a batch_size fitter given an LLM? Then we can send it to a database for a base value

# TODO: make it so that it stops at the start of the next input

Current: ['earthquake', 'aaalligatoorr']
Resulting Batch: 
AI: `earthquake`
`aaalligatoorr`
User 
END
Current: ['camp']
Resulting Batch: 
AI: `camp`
User 
END

OUTPUT: ['\nAI: `earthquake`\n`aaalligatoorr`\nUser ', '\nAI: `camp`\nUser ']
DONE

AI: `earthquake`
`aaalligatoorr`
User 
AI: `camp`
User 


In [8]:
import data_cleaning as dcl

print(dcl.raw_chain_inference(typofix_ptuner, propagated_tests))

NameError: name 'typofix_ptuner' is not defined

In [14]:
dcl.safe_chain_inference(typofix_ptuner, df["FixTypos"].unique())

'`cat`\n`hunter`\n`hunter`\n`'

In [19]:
len(' \nAI: `cat`\n`hunter`\n`hunter`\n`')

31